In [1]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk


def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 64))
            images.append(img)
            labels.append(label)
    return images, labels


genuine_images, genuine_labels = load_images_from_folder('path_to_genuine', 1)
forged_images, forged_labels = load_images_from_folder('path_to_forged', 0)


images = genuine_images + forged_images
labels = genuine_labels + forged_labels


def extract_hog_features(images):
    hog_features = []
    for img in images:
        fd, _ = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
        hog_features.append(fd)
    return np.array(hog_features)

features = extract_hog_features(images)


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)


class SignatureVerificationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Signature Verification System")
        self.root.geometry("500x400")
        self.root.configure(bg="#f0f0f0")

        self.label = tk.Label(root, text="Select an image or capture a live signature:", bg="#add8e6", fg="black", font=("Helvetica", 14))
        self.label.pack(pady=20)

        self.select_button = tk.Button(root, text="Select Image", command=self.select_image, bg="#008080", fg="white", font=("Helvetica", 12))
        self.select_button.pack(pady=10)

        self.capture_button = tk.Button(root, text="Capture Live Signature", command=self.capture_live_signature, bg="#008080", fg="white", font=("Helvetica", 12))
        self.capture_button.pack(pady=10)

        self.canvas_frame = tk.Frame(root, bg="#f0f0f0")
        self.canvas_frame.pack(pady=20)

        self.canvas = tk.Canvas(self.canvas_frame, width=128, height=64, bg="#d3d3d3")
        self.canvas.pack()

    def select_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.verify_signature(file_path)

    def capture_live_signature(self):
        cap = cv2.VideoCapture(0)
        cv2.namedWindow("Capture Signature")
        while True:
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Error", "Failed to capture image from webcam.")
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('Capture Signature', gray)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                img_path = "live_signature.png"
                cv2.imwrite(img_path, gray)
                cap.release()
                cv2.destroyAllWindows()
                self.verify_signature(img_path)
                break

    def preprocess_image(self, img):
        
        edges = cv2.Canny(img, 50, 150)
        return edges

    def validate_image(self, img):
      
        edges = self.preprocess_image(img)
        edge_density = np.sum(edges) / (128 * 64)
        return edge_density > 0.1 

    def verify_signature(self, file_path):
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 64))

            if not self.validate_image(img):
                messagebox.showinfo("Result", "This is not a valid signature image.")
                return

            fd, _ = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
            prediction = clf.predict([fd])
            if prediction[0] == 1:
                messagebox.showinfo("Result", "The signature is genuine.")
            else:
                messagebox.showinfo("Result", "The signature is forged.")
            self.display_image(img)
        else:
            messagebox.showerror("Error", "Failed to load the image.")

    def display_image(self, img):
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
        self.canvas.image = imgtk

if __name__ == "__main__":
    root = tk.Tk()
    app = SignatureVerificationApp(root)
    root.mainloop()
